In [1]:
!git clone https://github.com/chankiel/Steam-Review-NLP-Pipeline.git /content/Steam-Review-NLP-Pipeline


Cloning into '/content/Steam-Review-NLP-Pipeline'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 77 (delta 18), reused 67 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 1.52 MiB | 21.05 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [11]:
import sys, os

src_root = "/content/Steam-Review-NLP-Pipeline/src"
sys.path.insert(0, src_root)   # put it at the front

print("First 5 sys.path entries:")
print("\n".join(sys.path[:5]))

print("\nDoes src_root exist?", os.path.exists(src_root))
print("Contents of src_root:", os.listdir(src_root))

First 5 sys.path entries:
/content/Steam-Review-NLP-Pipeline/src
/content
/env/python
/usr/lib/python312.zip
/usr/lib/python3.12

Does src_root exist? True
Contents of src_root: ['training', 'inference', 'api', 'common', 'preprocess', 'utils']


In [12]:
preprocess_dir = "/content/Steam-Review-NLP-Pipeline/src/preprocess"

import os
print("Does preprocess dir exist?", os.path.exists(preprocess_dir))
if os.path.exists(preprocess_dir):
    print("Contents of preprocess dir:", os.listdir(preprocess_dir))

Does preprocess dir exist? True
Contents of preprocess dir: ['__init__.py', '__pycache__']


In [ ]:
import pandas as pd
import torch

from preprocess.group_summarizer import group_reviews
from utils.batching import batch_iter

from summarizer.pegasus_summarizer import PegasusSummarizer
from summarizer.textrank_summarizer import TextRankSummarizer
from summarizer.lstm_summarizer import LSTMSummarizer

INPUT_CSV = "/content/Steam-Review-NLP-Pipeline/data/processed/summarizer.csv"


ModuleNotFoundError: No module named 'preprocess.group_summarizer'

In [9]:
import importlib

print(importlib.import_module("preprocess.group_summarizer"))
print(importlib.import_module("summarizer.pegasus_summarizer"))


ModuleNotFoundError: No module named 'preprocess.group_summarizer'

In [ ]:
df = pd.read_csv(INPUT_CSV)

required_cols = {"app_id", "app_name", "review_text"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing columns: {missing}")

df.head()


In [ ]:
grouped = group_reviews(df)
len(grouped), grouped.head()


In [ ]:
import ipywidgets as widgets

model_selector = widgets.Dropdown(
    options=["pegasus", "textrank", "lstm"],
    value="pegasus",
    description="Model:"
)

model_selector


In [ ]:
from tqdm.notebook import tqdm

batch_size = 8
summaries = []

total_batches = len(grouped) // batch_size + 1

for idx_batch in tqdm(
    batch_iter(grouped.index.tolist(), batch_size),
    total=total_batches,
    desc="Summarizing"
):
    texts = grouped.loc[idx_batch, "clean_text"].tolist()
    batch_summaries = summarizer.summarize_batch(texts)
    summaries.extend(batch_summaries)


In [ ]:
output_path = f"../data/processed/game_summaries_{model_name}.csv"

grouped[f"summary_{model_name}"] = summaries
grouped.to_csv(output_path, index=False)

output_path
